In [1]:
from json.tool import main
from mimetypes import init
import math
import numpy as np
import scipy.spatial
from collections import Counter
from collections import OrderedDict
import pandas as pd #for data franes 
import matplotlib.pyplot as plt # for data visualization 
import seaborn as sns # for data visualization
import warnings
warnings.filterwarnings('ignore')
from sklearn.utils import shuffle
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [2]:
def dataframe(data):
        return pd.read_csv(data)

In [3]:
def shuff(data):
        return shuffle(data)

In [4]:
def ttsplit(data):
        X, y = train_test_split(data, test_size=0.2)
        return X,y

In [120]:
def entropy(target_col):
    val ,counts = np.unique(target_col,return_counts = True)
    entropy = 0
    for i in range(len(val)):
        entropy += (-counts[i]/np.sum(counts)) * np.log2(counts[i]/np.sum(counts))
    return entropy

In [121]:
def infogain(data, split_name, target_name = 'target'):
    total_entropy = entropy(data[target_name])
    print(total_entropy)
    vals,counts= np.unique(data[split_name],return_counts=True)
    average_entropy = 0
    for i in range(len(vals)):
        # print(i)
        attribute =  data.where(data[split_name]==vals[i]).dropna()[target_name]
        average_entropy  += (counts[i]/ np.sum(counts))* entropy(attribute)
    print(average_entropy)
    return total_entropy - average_entropy


In [122]:
data = 'votes.csv'
k = dataframe(data)
print(infogain(k, 'handicapped-infants'))

0.9623080486960709
0.8362349263792053
0.12607312231686563


In [39]:
a= 2
b= [ a/2 for i in range(10)]
print(b)
print(np.sum(b))
# print(np.sum([ a for i in range(10)]))

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
10.0
